In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

In [3]:
def date_time(table_cells):
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    out=''.join([booster_version for i,booster_version in enumerate(table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    out=[i for i in table_cells.strings][0]
    return out

def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass

def extract_column_from_header(row):
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
    colunm_name = ' '.join(row.contents)
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name

In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
response = requests.get(static_url)
soup = BeautifulSoup(response.text, 'html.parser')

### TASK 2: Extract all column/variable names from the HTML table header

In [5]:
html_tables = soup.find_all('table', "wikitable plainrowheaders collapsible")
first_launch_table = html_tables[2]
column_names = []
for th in first_launch_table.find_all('th'):
    name = extract_column_from_header(th)
    if name:
        column_names.append(name)
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


In [6]:
launch_dict = dict.fromkeys(column_names)
del launch_dict['Date and time ( )']
for key in launch_dict.keys():
    launch_dict[key] = []

In [8]:
# Créer un dictionnaire pour stocker les données
launch_dict = {
    'Flight No.': [],
    'Launch site': [],
    'Payload': [],
    'Payload mass': [],
    'Orbit': [],
    'Customer': [],
    'Launch outcome': [],
    'Version Booster': [],
    'Booster landing': [],
    'Date': [],
    'Time': []
}

In [10]:
extracted_row = 0
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    for rows in table.find_all("tr"):
        if rows.th and rows.th.string and rows.th.string.strip().isdigit():
            flight_number = rows.th.string.strip()
            row = rows.find_all('td')
            if row:
                datatimelist = date_time(row[0])
                launch_dict['Flight No.'].append(flight_number)
                launch_dict['Date'].append(datatimelist[0].strip(','))
                launch_dict['Time'].append(datatimelist[1])
                launch_dict['Version Booster'].append(booster_version(row[1]))
                launch_dict['Launch site'].append(row[2].a.string if row[2].a else None)
                launch_dict['Payload'].append(row[3].a.string if row[3].a else None)
                launch_dict['Payload mass'].append(get_mass(row[4]))
                launch_dict['Orbit'].append(row[5].a.string if row[5].a else None)
                launch_dict['Customer'].append(row[6].a.string if row[6].a else None)
                launch_dict['Launch outcome'].append(list(row[7].strings)[0] if row[7].strings else None)
                launch_dict['Booster landing'].append(landing_status(row[8]))

In [11]:
df = pd.DataFrame({key: pd.Series(value) for key, value in launch_dict.items()})
df.to_csv('spacex_web_scraped.csv', index=False)